# INSIDE RSA

(i) Computing the gcd; (ii) Computing the modular multiplicative inverse (when it exists); (iii) Encryption/Decryption with Textbook RSA; (iv) Signature with Full-Domain Hash

In [1]:
from functools import reduce
import random
from Crypto.Util import number
from hashlib import sha256
from base64 import b64encode, b64decode


(i) Computing the gcd

#### NEW GCD

In [2]:
#Implement the Greatest Common Divisor (gcd) for any list of numbers

''' 
a,b: A list of numbers given as an input

gcd function:
In case we have 2 numbers as an input, we return
the (greatest) number until the division between him 
and the other has zero remainder.

Output : return (g, x, y) such that a*x + b*y = g = gcd(x, y)

'''

def egcd(a, b):
    if a == 0:
        return b, 0, 1
    else:
        g, y, x = egcd(b % a, a)
        return g, x - (b // a) * y, y



In [4]:
a = 4
b = 2
print('the gcd of',a,'and',b,'is',egcd(a,b)[0])


the gcd of 4 and 2 is 2


(ii) Computing the modular multiplicative inverse (when it exists)

#### INVERSE

In [3]:
'''
Via the Extended Euclidean GCD algorithm
p : a positive integer (the divisor)
q : a positive integer (the dividend)
Given (p,q) we compute the inverse of x if it exists.

'''
def inverse_rsa(p,q):
    g, x, y = egcd(p, q)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % q

In [7]:
p = 9
q = 7
print('The modular multiplicative inverse of',p,'and',q,'is',inverse_rsa(9,7))

The modular multiplicative inverse of 9 and 7 is 4


In [24]:
#a = 108 ** 151
#m = 22499

In [25]:
#%timeit inverse_rsa (a ,m)

 (iii) Encryption/Decryption with Textbook RSA

INPUT: Security parameter l
OUTPUT: RSA public key e, private key d and n
1. Randomly select two primes p and q with same bitlength l/2
2. Compute n = pq and phi = (p-1)(q-1)
3. Select an arbitrary integer e with 1 < e < phi and gcd(e, phi)==1
4. Compute the integer d satisfying 1 < d < phi and ed == 1 mod phi
5. Return(n, e, d)


Algorithm 1.2: Basic RSA encryption
INPUT: RSA public key e, n, plaintext m
OUTPUT: Ciphertext c
1. Compute c = m**e mod n
2. Return(c)
Algorithm 1.3: Basic RSA decryption
INPUT: RSA private d, n, ciphertext c
OUTPUT: Plaintext m
1. Compute m = c**d mod n
2. Return(m)

In [4]:
#Generate random prime numbers with specific length of bits

def prime_gen(bit_length):
    #random.seed(123)
    bits = bit_length//2
    prime = number.getPrime(bits)
    return (prime)

### Construct the φ(Ν) function

In [5]:
#We choose the encryption "e" such that gcd(e,φ(N))=1
'''
Input : the value of "φ" function
Output : the encryption "e"
We randomly pick a candidate number less than φ(Ν)
and check if the gcd(e,φ(N))=1
'''


def encrypt_pow(phi_N):
    while True:
        e = random.randrange(1,phi_N)
        if egcd(e, phi_N)[0] == 1:
            return(e)
            break
                 
             

In [27]:
encrypt_pow(15)

2

In [6]:
# Generate the public key and the private key

'''
Input : number of bits which will define the prime bitlength
Output: The primes (p,q),their product (N) and the function phi_N

'''
def Key_Generation(bits):
    # Randomly select 2 primes with same Bitlength l/2
    p = prime_gen(bits)
    q = prime_gen(bits)
    # Compute
    N = p * q
    phi_N = (p - 1) * (q - 1)
    # Select an arbitrary integer e with 1 < e < phi and gcd(e,phi) == 1
    e = encrypt_pow(phi_N)
    # Compute the integer d statisfying 1 < d < phi and e*d == 1 mod phi
    d = inverse_rsa(e, phi_N)
    # Return n e d
    print("Public Key: " ,e)
    print("Private Key: " ,d)
    print("N =",N)
    return(p,q,N,phi_N,e,d)

In [7]:
prime_gen(600)

1059755015600927137567066027083281716411542081080556558775129937888706700351506844678565879

In [482]:
Key_Generation(64)

Public Key:  1876205324478243827
Private Key:  3699851857851622175
N = 10078627333082037833


(3598116187,
 2801084459,
 10078627333082037833,
 10078627326682837188,
 1876205324478243827,
 3699851857851622175)

RSA Encryption Scheme

Input: The plaintext m,the RSA public key e and n
    
Output: The ciphertext c
    
1. Compute c = m^e mod n
2. Return(c)

In [15]:
'''
Input: Encrypt a message m, with public key e using modulus N
Output: Return the ciphertext : c=m^e modN
'''

def encryption(N,e,m):
    
    return (expm(m, e, N))

In [14]:

'''
Input: Decrypt a received cipher c, with private key d using modulus N
Output: Return the plaintexnt : m=c^d modN
'''

def decryption(N,d,c):
    return expm(c, d, N)
    


https://www.youtube.com/watch?v=EHUgNLN8F1Y
    FOR SQUARE AND MULTIPLY

In [486]:
'''
We use the square and multiply method to do the modular exponentiation
Returns: x**y modulo z.

'''
def ex(x, y,z):
    exp = bin(y)
    value = x
 
    for i in range(3, len(exp)):
        value = (value * value) %z
        if(exp[i:i+1]=='1'):
            value = (value*x) % z
    return value 

In [10]:
'''
We use the square and multiply method to do the modular exponentiation
Return: a**k modulo N.
'''

def expm(a, k, N):
    r = 1
    bits = list(bin(k)[2:])
    for bit in bits:
        r = (r * r) % N
        if int(bit) == 1:
            r = (r * a) % N
    return r

In [353]:
expm(3,200,50)

1

### Convert string into number

In [2]:
strg = 'hey'
i = int.from_bytes(strg.encode('utf-8'), byteorder='big')

### Convert number into string

In [3]:
s = int.to_bytes(i, length=len(strg), byteorder='big').decode('utf-8')


RSA Decryption Scheme

Input:The ciphertext c,the RSA private key d, and  N
    
Output: The plaintext m
    
1. Compute m = c**d mod n
2. Return(m)

Decrypt and get the original message (plaintext)

(iv) Signature with Full-Domain Hash

In [11]:
#Hash the messagewith SHA-256

'''
Input : message (the plaintext)
Output : The hashed message
Procedure : To generate a signature, make a hash from the plaintext, 
encrypt it with your private key, include it alongside the plaintext.
'''



def hashFunction(message):
    hashed = sha256(message.encode("UTF-8")).hexdigest()
    return hashed


In [12]:
#Authentication Procedure

'''
Input : receivedeHash (the hash we received after decryption),message (the plaintext)
Output : The result of verification 

Procedure : Make a hash from the plaintext (ourHashed), decrypt the signature 
with the sender's public key (decrypted_msg),
check that both hashes are the same (receivedHash ~ ourHashed)
'''



def verify(receivedHash, message):
    Hash_it = hashFunction(message)
    ourHash = int.from_bytes(Hash_it.encode('utf-8'), byteorder='big')
    if receivedHash == ourHash:
        print(1)
        print("Authentication successful: ", )
        print(receivedHash, " = ", ourHash)
    else:
        print(0)
        print("Authentication failed")
        print(receivedHash, " != ", ourHash)
    


Main function

Input : Number of bits which define the bitlength (bits), the plaintext (message)
Output : The public-private generated key pair, the encrypted-decrypted message, the results of the verification process



In [22]:
def main():
    bits = int(input("Specify the number of bits for the prime generation:"))
    message = str(input("Enter a message to encrypt with your private key: "))
    decoded_message = int.from_bytes(message.encode('utf-8'), byteorder='big')
    #Prime Generation,private and public key
    p = prime_gen(bits)
    q = prime_gen(bits)
    N = p * q
    phi_N = (p - 1) * (q - 1)
    e = encrypt_pow(phi_N)
    d = inverse_rsa(e, phi_N)
    print("")
    print("Public Key: " ,e)
    print("")
    print("Private Key: " ,d)
    print("")
    print("N =",N)
    #Encryption and Decryption
    c = encryption(N,e,decoded_message)
    print("")
    print("The encrypted message with public key e =",e,"is")
    print("")
    print("c = ",c)
    print("")
    print("The dectrypted message with private key d =",d,"is")
    print("")
    dec = decryption(N,d,c)
    print(int.to_bytes(dec, length=len(message), byteorder='big').decode('utf-8'))  
    #Sign the message and verify
    hashed = hashFunction(message)
    enc = int.from_bytes(hashed.encode('utf-8'), byteorder='big')
    print("")
    print("Encrypting message with private key d = ", d ," . . .")
    encrypted_msg = encryption(N,d,enc)  
    print("Your signature is : tag =")
    print(encrypted_msg)
    print("")
    print("Decrypting message with public key e = ", e ," . . .")
    decrypted_msg = decryption(N,e,encrypted_msg)
    print("Your decrypted message is: tag =")  
    print(decrypted_msg)
    print("")
    print("Verification results~")
    message = str(input("Enter a message to encrypt with your private key: "))
    verify(decrypted_msg, message)


main()    

Specify the number of bits for the prime generation:3000
Enter a message to encrypt with your private key: RSA is one of the first public-key cryptosystems and is widely used for secure data transmission. In such a cryptosystem, the encryption key is public and it is different from the decryption key which is kept secret (private).

Public Key:  2877163399001285062396035342469237559854533721784020708611174843159930383060729752542256887143828136539794652737926925388565865458771229037541317470483887812945129175901366995394419453992279382696932616065513489658970756007309437881634829956537552343585141418463815688362262867298385041717186740500762496789342078453260327858230087288940728217617337934163662363177348745110807443878984603246479082723274643551393012623826196281652820574860958763891795599105897040916371192272686720588130884154815221665736623081953680344804646357706894926528887099456997177707451827230577414542630790504043389211852910360321815963804796388465582512169710850513328056610

Enter a message to encrypt with your private key: RSA is one of the first public-key cryptosystems and is widely used for secure data transmission. In such a cryptosystem, the encryption key is public and it is different from the decryption key which is kept secret (private).
1
Authentication successful: 
5143562095290045369404561511587114621500792432118968750002934526671284204669733418185042325721064542199251338849844672105513504151907338976368952533463857  =  5143562095290045369404561511587114621500792432118968750002934526671284204669733418185042325721064542199251338849844672105513504151907338976368952533463857
